# 크롤링을 통해 받아온 데이터를 1차적으로 튜닝한 프로세스

In [659]:
import numpy as np
import pandas as pd
from IPython.display import display
import re

In [660]:
df = pd.read_csv('references.csv')

In [661]:
new_df = pd.read_csv('extended_references.csv')

In [662]:
only_infor_tuned_df = pd.read_csv('tuned_dataframe_infor.csv')

# 기존에 시행했던 데이터 프레임의 튜닝 방식이 혼재되어 있다고 판단

# 길이에 따라서 (1,3,4,5) 분류하고, 논문의 특성에 따라서 재 분류하여

# 해당 논문이 갖춰야 할 정보들을 충족하는 논문 즉, 딕셔너리에 쓰일 Cited paper 를 

# 보다 효율적으로 찾아내는 프로세스를 진행한다.

In [663]:
new_df.tail()

,infor,refer
37242,"Corporate Diversification, Relatedness, and Fi...","40. [기타자료] - Villalonga, Belen / 1999 / Does D..."
37243,"Corporate Diversification, Relatedness, and Fi...","41. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ..."
37244,"Corporate Diversification, Relatedness, and Fi...","42. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ..."
37245,"Corporate Diversification, Relatedness, and Fi...","43. [학술지(정기간행물)] - Weston, J. F / 1970 / The N..."
37246,"Corporate Diversification, Relatedness, and Fi...","44. [학술지(정기간행물)] - Whited, Toni M / 2001 / Is ..."


/ 자르면 일단 어느정도는 잘린다.

In [664]:
new_refer_ls = []

for idx in range(len(new_df['refer'].values)):
    new_refer_ls.append(new_df['refer'].values[idx].split(' / '))
len(new_refer_ls) == len(new_df)

True

In [665]:
new_df['tuned_refer'] = new_refer_ls
new_df.tail()

,infor,refer,tuned_refer
37242,"Corporate Diversification, Relatedness, and Fi...","40. [기타자료] - Villalonga, Belen / 1999 / Does D...","[40. [기타자료] - Villalonga, Belen, 1999, Does Di..."
37243,"Corporate Diversification, Relatedness, and Fi...","41. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ...","[41. [학술지(정기간행물)] - Villalonga, Belen, 2004, D..."
37244,"Corporate Diversification, Relatedness, and Fi...","42. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ...","[42. [학술지(정기간행물)] - Villalonga, Belen, 2004, D..."
37245,"Corporate Diversification, Relatedness, and Fi...","43. [학술지(정기간행물)] - Weston, J. F / 1970 / The N...","[43. [학술지(정기간행물)] - Weston, J. F, 1970, The Na..."
37246,"Corporate Diversification, Relatedness, and Fi...","44. [학술지(정기간행물)] - Whited, Toni M / 2001 / Is ...","[44. [학술지(정기간행물)] - Whited, Toni M, 2001, Is I..."


In [666]:
set([len(i) for idx,i in enumerate(new_df['tuned_refer'])])


{1, 3, 4, 5}

/ 로 잘라낸 리스트 별로 길이가 다르다. 이는 kci에서 제공하는 정보는 / 별로 성질이 나뉘어 지므로, 논문 별로 제공하는 정보의 내용과 양이 나르거나, 특성이 다르다고 판단하여 길이 별로 들여다본다.

In [667]:
print(len([i for i in new_refer_ls if len(i) == 1]),
len([i for i in new_refer_ls if len(i) == 3]),
len([i for i in new_refer_ls if len(i) == 4]),
len([i for i in new_refer_ls if len(i) == 5]))

242 8071 28840 94


# 나중에 병합할 때 혼류가 없도록 길이에 따른 인덱스를 저장해준다

In [668]:
len_1_idx = [idx for idx,val in enumerate(new_df['tuned_refer']) if len(val) == 1] # 없는 값
len_3_idx = [idx for idx,val in enumerate(new_df['tuned_refer']) if len(val) == 3]
len_4_idx = [idx for idx,val in enumerate(new_df['tuned_refer']) if len(val) == 4]
len_5_idx = [idx for idx,val in enumerate(new_df['tuned_refer']) if len(val) == 5]
len(len_1_idx),len(len_3_idx),len(len_4_idx),len(len_5_idx)

(242, 8071, 28840, 94)

In [669]:
len(len_1_idx) +len(len_3_idx) +len(len_4_idx)  + len(len_5_idx) == len(new_df)

True

# 논문의 특성을 나타내는 첫 번째 리스트와 이름이 결합되어 있는 경우가 발생, split 해준다.

\- 기준으로 split

In [670]:
tuned_new_refer = []
for idx in range(len(new_refer_ls)):
    tuned_new_refer.append(new_refer_ls[idx][0].split(" -") + new_refer_ls[idx][1:])
len(tuned_new_refer) == len(new_refer_ls)

True

가장 첫 번째 character 엘리먼트와 3번째 엘리먼스인 년도 사이에 무엇이 있으면 하나로 병합시켜준다. (이름)

In [671]:
merge = [(1, 3)]

for ls in tuned_new_refer : 
    if ls[0] != '0': 
        if not re.findall('[0-9]{1,4}',ls[2]) : 
            for t in merge[::-1]:
                merged = ''.join(ls[t[0]:t[1]])  # merging values within a range
                ls[t[0]:t[1]] = [merged]

아무것도 안나오면 정상

In [672]:
for ls in tuned_new_refer : 
    if ls[0] != '0': 
        if not re.findall('[0-9]{1,4}',ls[2]) : 
            print(ls)

1을 제외하고 나머지는 1씩 증가했다.

In [673]:
set([len(i) for i in tuned_new_refer])

{1, 4, 5, 6}

In [674]:
len_1_idx = [idx for idx,val in enumerate(new_df['tuned_refer']) if len(val) == 1] # 없는 값
len_3_idx = [idx for idx,val in enumerate(new_df['tuned_refer']) if len(val) == 4]
len_4_idx = [idx for idx,val in enumerate(new_df['tuned_refer']) if len(val) == 5]
len_5_idx = [idx for idx,val in enumerate(new_df['tuned_refer']) if len(val) == 6]
len(len_1_idx),len(len_3_idx),len(len_4_idx),len(len_5_idx)

(242, 28840, 94, 0)

In [675]:
new_df['tuned_refer'] = tuned_new_refer

In [676]:
new_df['length_refer'] = new_df['tuned_refer'].apply(lambda x : len(x))

In [677]:
new_df.tail()

,infor,refer,tuned_refer,length_refer
37242,"Corporate Diversification, Relatedness, and Fi...","40. [기타자료] - Villalonga, Belen / 1999 / Does D...","[40. [기타자료], Villalonga, Belen, 1999, Does Di...",4
37243,"Corporate Diversification, Relatedness, and Fi...","41. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ...","[41. [학술지(정기간행물)], Villalonga, Belen, 2004, D...",5
37244,"Corporate Diversification, Relatedness, and Fi...","42. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ...","[42. [학술지(정기간행물)], Villalonga, Belen, 2004, D...",5
37245,"Corporate Diversification, Relatedness, and Fi...","43. [학술지(정기간행물)] - Weston, J. F / 1970 / The N...","[43. [학술지(정기간행물)], Weston, J. F, 1970, The Na...",5
37246,"Corporate Diversification, Relatedness, and Fi...","44. [학술지(정기간행물)] - Whited, Toni M / 2001 / Is ...","[44. [학술지(정기간행물)], Whited, Toni M, 2001, Is I...",5


In [678]:
set(new_df['length_refer'])

{1, 4, 5, 6}

In [679]:
def show_me_the_list_under_char_and_length(length_ , char,how_much=None):
    length_detect_ls = new_df[new_df['length_refer'] == length_].index.tolist()
    char_detect_ls = \
    [idx for idx,i in enumerate(new_df['tuned_refer'].tolist()) if re.findall('\[.+\]',i[0]) == ['[{}]'.format(char)]]
    return new_df.loc[list(set(length_detect_ls).intersection(char_detect_ls)),:]['tuned_refer'].tolist()[:how_much]

# 4개짜리의 구조
- 1. 논문의 성향
- 2. 논문의 저자(없는 경우 공백 : '')
- 2. 논문의 년도
- 3. 논문의 이름

In [680]:
Counter([re.findall('\[.+\]',i)[0] for i in [i[0] for i in new_df[new_df['length_refer'] == 4]['tuned_refer'].tolist()]])

Counter({'[기타자료]': 348,
         '[단행본]': 58,
         '[보고서]': 2027,
         '[학술대회논문]': 2,
         '[학술지(정기간행물)]': 5636})

### 길이가 4개인 리스트에서 카테고리가 보고서인 경우
- 보고서가 완전한 모양새를 갖추기 위해서는 카테고리 , 저자명 , 저널이름 , 논문 이름 , 이슈 년도가 있어야 한다.
- 길이가 4개인 경우, 이를 다 갖추지 못하고, 단지 이름과 년도 논문 이름만 나와있는 것을 볼 수 있다.

In [681]:
show_me_the_list_under_char_and_length(4,'보고서',5)

[['49. [보고서]',
  ' Scharfstein, D. S',
  '1998',
  'The dark side of internal capital markets：Evidence from dividend conglomerates,'],
 ['28. [보고서]', ' Sautner, Z', '2005', 'Stock options and employee behavior'],
 ['16. [보고서]',
  ' Chemmanur, T. J',
  '2011',
  'How does venture capital financing improve efficiency in private firms? A look beneath the surface,'],
 ['5. [보고서]',
  ' Bruche, M',
  '2015',
  'Debt maturity and the liquidity of secondary markets,'],
 ['6. [보고서]',
  ' Burlon, L',
  '2016',
  'The Quantity of Corporate Credit Rationing with Matched Bank-Firm Data,']]

In [682]:
[i[-1][:-1] for i in show_me_the_list_under_char_and_length(4,'보고서') if ',' in i[-1][:-1]][:10]

['Bank Consolidation, Internationalization and Conglomeration：Trends and Implications for Financial Risk',
 'Risk Measurement, Risk Management and Capital Adequacy in Financial Conglomerates',
 'The SME ‘Finance Gap’: Myth or Reality?, Research report on SME attitudes and experiences with bank loan applications',
 'Loss Given Default: The Link between Default and Recovery Rates, Recovery Ratings and Recent Empirical Evidence',
 'Credit-Rating Shopping, Selection and the Equilibrium Structure of Ratings',
 'Trading Analysis of October 27 and 28, 1997: A Report by the Division of Market Regulation to the U.S. Securities and Exchange Commission',
 'Tobin’s Q Does Not Measures Firm Performance:Theory, Empirics, and Alternative Measure’',
 '호가, 체결 자료를 이용한 외국인 주식투자행태 연구',
 'Measuring the financial soundness of US firms, 1926~2012',
 'Earnings volatility, earnings management andequity value']

# 길이가 4개인 보고서
- 어쩌구 : 저쩌구 
    - 둘 중 하나가 저널명인지에 대해 확인해본 결과, 본제 부제의 관계임을 확인할 수 있었다.
- 쉼표 , 기준
    - 특정한 기준은 없지만 working paper 가 산재되어 있다.

### 길이가 4개인 리스트에서 카테고리가 단행본인 경우

- 단행본가 완전한 모양새를 갖추기 위해서는 카테고리 , 저자명 , 저널이름 , 논문 이름 , 이슈 년도가 있어야 한다.
- 길이가 4개인 경우, 이를 다 갖추지 못하고, 단지 이름과 년도 논문 이름만 나와있는 것을 볼 수 있다.

In [683]:
show_me_the_list_under_char_and_length(4,'단행본',5)

[['6. [단행본]', ' 삼성전자', '2010', '삼성전자 40년: 도전과 창조의 역사,'],
 ['105. [단행본]',
  ' Ongena, S',
  '2000',
  'The Performance of Financial Institutions,'],
 ['44. [단행본]',
  ' Schumpeter, J. A.',
  '1912',
  'Theorie der wirtschaftlichen Entwicklung,'],
 ['81. [단행본]',
  ' Schwert, G. W',
  '2003',
  'Anomalies and Market Efficiency , In Handbook of the Economics ofFinance : 939 ~ 974,'],
 ['20. [단행본]',
  ' Eckbo, E',
  '2007',
  'Handbook of Corporate Finance: Empirical Corporate Finance 1 : 233 ~ 373,']]

In [684]:
[i[-1][:-1] for i in show_me_the_list_under_char_and_length(4,'단행본') if ',' in i[-1][:-1]]

['Anomalies and Market Efficiency , In Handbook of the Economics ofFinance : 939 ~ 974',
 'Early warning model for bank supervision: Simpler could be better, Economic Respective (FRB of Chicago), 3rd Quarter : 49 ~ 60',
 'Small Business Lending, In The Oxford Handbook of Banking',
 'Panel data, in Z. Griliches and M. Intrilligator ; Handbook of Econometrics, 2',
 'iskMetrics-Technical Document, 4th Edition',
 'The Financial Accelerator in a Quantitative Business Cycle Framework, In Handbook of Macroeconomics, Vol. 1C : 1341 ~ 1393',
 'Exchange Traded Funds : History, Trading and Research, in Handbook ofFinancial Engineering, Zopounidis, Constantin, Michael Doumpos, andPanos : 1 ~ 37',
 'Stock Pyramids, Cross-ownership, and Dual Class Equity in Concentrated Corporate Ownership : 295 ~ 315',
 'The Structure and Performance of The Money Management Industry, In Brookings Papers on Economic Acitvity : Microeconomics : 339 ~ 391',
 'Teh world of Mathematics, 2',
 'Small Business Lending, In 

# 길이가 4개인 단행본
- 어쩌구 : 저쩌구 
    - 둘 중 하나가 저널명인지에 대해 확인해본 결과, 본제 부제의 관계임을 확인할 수 있었다.
- 쉼표 , 기준
    - 특정한 기준은 없지만 Handbook , Papers 등이 산재되어 있다.

### 길이가 4인 리스트에서 카테고리가 기타 자료인 경우
- 단행본이 모양새를 다 갖추기 위해서는,카테고리 , 저자명 , 저널이름 , 논문이름 , 이슈 년도가 있어야 한다.
- 위의 기타 자료와 단행본 과는 다르게, 저널 명이 보이는 경우가 있다.(Manuscript , 괄호안에 있는 기관들 또는 워킹 페이퍼 , SSRN)
- 패턴은 지켜본 결과, 쉼표로 이어져 있거나 괄호 안에 위치해 있는 경우가 있다.

In [685]:
show_me_the_list_under_char_and_length(4,'기타자료',5)

[['40. [기타자료]',
  ' Kim, Yun-Yeong',
  '2009',
  'Dynamic Analyses for Cointegration Disequilibrium Error via Transformed VAR Model, Manuscript,'],
 ['66. [기타자료]', ' Korea Exchange', '2010', '2009 KRX Fact Book,'],
 ['15. [기타자료]', ' 한국기업평가', '2014', 'CP Rating Performance,'],
 ['16. [기타자료]', ' 한국신용평가', '2014', 'CP Rating Performance, 2013,'],
 ['3. [기타자료]',
  ' Bowers Jr., N. L',
  '1997',
  'Actuarial Mathematics (The Society of Actuaries, United States of America),']]

In [686]:
[i[-1][:-1] for i in show_me_the_list_under_char_and_length(4,'기타자료') if ',' in i[-1][:-1]][:10]

['Dynamic Analyses for Cointegration Disequilibrium Error via Transformed VAR Model, Manuscript',
 'CP Rating Performance, 2013',
 'Actuarial Mathematics (The Society of Actuaries, United States of America)',
 'The PCS option, an improvement of the CAT-future (Manuscript)',
 'orporate Governance and the Cost of Equity Capital, Working paper, SSRN',
 'Firm Level Analysis of the Korean Corporate Sector: 1996～2000. in International Monetary Fund. Republic of Korea: Selected Issues, IMF Country Report No. 02/20',
 'Earnings Quality, Research Foundation of CFA Institute',
 'Firms’ Capital Allocation Choices, Information Quality, and the Cost of Capital, Working paper',
 'Michaely, 2002, Payout Policy, Working paper, The Wharton School, University of Pennsylvania',
 'The Conditional nature of the value of corporate governance, Financial Management Association 2005, Chicago, Illinois']

# 길이가 4개인 기타 자료
- 어쩌구 : 저쩌구 
    - 둘 중 하나가 저널명인지에 대해 확인해본 결과, 본제 부제의 관계임을 확인할 수 있었다.
- 쉼표 , 기준
    - 특정한 기준은 없지만 Manuscripts , Report 등이 산재되어 있다. (괄호 안에 있는 경우가 다수 있다.)

### 길이가 4인 리스트에서 카테고리가 학술 대회 논문인 경우
- 학술대회논문이 모양새를 다 갖추기 위해서는,카테고리 , 저자명 , 학교이름 , 학위 이름, 논문이름 , 이슈 년도가 있어야 한다.
- 길이가 4개인 경우, 이를 다 갖추지 못하고, 저자 이름 / 년도 / 논문 이름만 기제되어 있는 것을 알 수 있다.

In [687]:
show_me_the_list_under_char_and_length(4,'학술대회논문')

[['15. [학술대회논문]',
  ' Standard',
  '2009',
  '/ 2008 Annual global corporate default study and rating tran sitions,'],
 ['12. [학술대회논문]', ' 정병선', '2008', '한국 상장기업의 파생상품 사용 결정요인,']]

In [688]:
[i[-1][:-1] for i in show_me_the_list_under_char_and_length(4,'학술대회논문')]

['/ 2008 Annual global corporate default study and rating tran sitions',
 '한국 상장기업의 파생상품 사용 결정요인']

# 길이가 4인 학술 대회 논문
- 저널/기관 명이 없는 데이터가 2개 뿐이므로 날려버린다.

### 길이가 4인 리스트에서 카테고리가 학술지인 경우
- 학술대회논문이 모양새를 다 갖추기 위해서는,카테고리 , 저자명 , 학교이름 , 학위 이름, 논문이름 , 이슈 년도가 있어야 한다.
- 길이가 4개인 경우, 이를 다 갖추지 못하고, 저자 이름 / 년도 / 논문 이름만 기제되어 있는 것을 알 수 있다.

In [689]:
show_me_the_list_under_char_and_length(4,'학술지(정기간행물)',5)

[['1. [학술지(정기간행물)]',
  '',
  '2001',
  '은행중심 자금조달 시스템의 문제점: 은행의 유동성위기가 기업가치에 미치는 영향 14 (1) : 161 ~ 188,'],
 ['2. [학술지(정기간행물)]', '', '2000', '국내 은행간 합병의 효과분석 17 (1) : 283 ~ 306,'],
 ['3. [학술지(정기간행물)]', '', '2000', '은행퇴출과 관계효과 상실에 관한 실증연구 5 (3) : 1 ~ 27,'],
 ['4. [학술지(정기간행물)]',
  '',
  '2002',
  '관계금융효과의 비대칭성에 관한 실증연구: 제일은행 사례를 중심으로 : 425 ~ 463,'],
 ['5. [학술지(정기간행물)]',
  '',
  '1999',
  'The Consolidation of the Financial Services Industry: Causes, Consequences, and Implications for the Future 23 : 135 ~ 194,']]

In [690]:
print(len([i[-1][:-1] for i in show_me_the_list_under_char_and_length(4,'학술지(정기간행물)') if 'Journal' in i[-1][:-1] ]))
[i[-1][:-1] for i in show_me_the_list_under_char_and_length(4,'학술지(정기간행물)') if 'Journal' in i[-1][:-1] ][:10]

951


['The Benefits of Lending Relationships Evidence from Small Business Data Journal of Finance 49 : 3 ~ 37',
 'A Stylized Model of Customer Relationships Journal of Finance 45 : 1069 ~ 1087',
 'The Value of Bank Durability Borrowers as Bank Stakeholders Journal of Finance 48 : 247 ~ 266',
 'Valuation of primary issue convertible bonds Journal of Financial Research 9 : 251 ~ 259',
 'The case of event studies Journal of Financial Economics 8 : 1 ~ 31',
 'Journal of Financial Economics 13 : 115 ~ 136',
 'P/E Ratio and Earnings Capitalization Under Uncertainty Journal of Accounting Research : 38 ~ 46',
 'Journal of Financial Economics : 263 ~ 292',
 'The Role of Volume Journal of Finance (49) : 153 ~ 181',
 'Returns to Buying Winners and Selling Losers Implication for Stock Market Efficiency Journal of Finance : 65 ~ 91']

# 길이가 4인 학술지 정기간행물
- 이 부분이 제일 더럽다. 위의 리스트는 Journal 로 검색했을 때의 결과물이다. -- 951개

# # # ## # # # # # # # # # # # #  # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# 길이가 5개인 경우

In [691]:
Counter([re.findall('\[.+\]',i)[0] for i in [i[0] for i in new_df[new_df['length_refer'] == 5]['tuned_refer'].tolist()]])

Counter({'[기타자료]': 214,
         '[단행본]': 1026,
         '[학술대회논문]': 383,
         '[학술지(정기간행물)]': 27212,
         '[학위논문]': 5})

### 길이가 5인 리스트에서 카테고리가 단행본인 경우
- 기타 자료가 모양새를 다 갖추기 위해서는,카테고리 , 저자명 , 저널명 , 논문이름 , 이슈 년도가 있어야 한다.
- 아래에서 5개만 끊어서 보면, 어느정도 구색을 맞추고 있는 것으로 보인다. 자세히 살펴보아서 모두 완벽한지 확인해봐야 한다.

In [692]:
show_me_the_list_under_char_and_length(5,'단행본',5)

[['14. [단행본]',
  ' 이광배',
  '2011',
  'ELS 상품의 손익구조와 헤지운용 : 이론과 실제(step down autocall 구조를 중심으로',
  '하나금융투자,'],
 ['13. [단행본]',
  ' Hastie, T. J.',
  '1990',
  'Generalized Additive Models',
  'Chapman and Hall,'],
 ['17. [단행본]', ' 이효섭', '2013', '한국 ELS/DLS 시장의 건전한 성장을 위한 방안', '자본시장연구원,'],
 ['28. [단행본]',
  ' Friedman, M',
  '1963',
  'A Monetary History of the United States, 1867～1960',
  'Princeton University Press,'],
 ['31. [단행본]',
  ' Garber, P. M.',
  '2000',
  'Famous First Bubbles: the Fundamentals of Early Manias',
  'MIT Press,']]

In [693]:
[i[1] for i in show_me_the_list_under_char_and_length(5,'단행본') if not i[1]]

['', '']

In [694]:
[i[-1][:-1] for i in show_me_the_list_under_char_and_length(5,'단행본') if not i[-1][:-1]]

[]

# 길이가 5인 단행본

- 저널명은 공백이 없는 것으로 보아 큰 문제가 없어보인다.
- 이름이 공백인 경우가 2가지 있었다.
    - 이 경우는 딕셔너리에서 제외

### 길이가 5인 리스트에서 카테고리가 기타 자료인 경우
- 기타 자료가 모양새를 다 갖추기 위해서는,카테고리 , 저자명 , 저널명 , 논문이름 , 이슈 년도가 있어야 한다.
- 아래에서 5개만 끊어서 보면, 어느정도 구색을 맞추고 있는 것으로 보인다. 자세히 살펴보아서 모두 완벽한지 확인해봐야 한다.

In [695]:
show_me_the_list_under_char_and_length(5,'기타자료',5)

[['10. [기타자료]',
  ' Chakravarty, S',
  '1998',
  'Information asymmetry, market segmentation and pricing d cross-listed shares: Theory and evidence from Chinese A and B shares',
  'Working paper, Purdue Univ,'],
 ['3. [기타자료]',
  ' Lydienne Matchie',
  '2009',
  'Malliavin Calculus and Some Applications in Finance',
  'African Institute for Mathematical Sciences,'],
 ['6. [기타자료]', ' 한국예탁결제원', '2012', '보호예수제도 실무안내', '한국예탁결제원,'],
 ['10. [기타자료]',
  ' Euromoney',
  '2007',
  'The 2007 Guide to Financial Supply-Chain Management',
  'HSBC,'],
 ['11. [기타자료]',
  ' Feinberg, S.',
  '2007',
  'So You Think You Understand Supply Chain Finance: A Primer on Innovative Working Capital Financing',
  'TowerGroup,']]

In [696]:
display([i[1] for i in show_me_the_list_under_char_and_length(5,'기타자료') if not i[1]])
len([i[1] for i in show_me_the_list_under_char_and_length(5,'기타자료') if not i[1]])

['', '', '', '', '', '', '', '', '', '', '']

11

In [697]:
[i[-1][:-1] for i in show_me_the_list_under_char_and_length(5,'기타자료') if not i[-1][:-1]]

[]

# 길이가 5인 기타 자료

- 저널명은 큰 문제가 없어보인다.
- 이름이 공백인 경우가 11가지 있었다.
    - 이 경우는 딕셔너리에서 제외

### 길이가 5인 리스트에서 카테고리가 학술대회논문인 경우
- 학술대회논문가 모양새를 다 갖추기 위해서는,카테고리 , 저자명 , 저널명 , 논문이름 , 이슈 년도가 있어야 한다.
- 아래에서 5개만 끊어서 보면, 어느정도 구색을 맞추고 있는 것으로 보인다. 자세히 살펴보아서 모두 완벽한지 확인해봐야 한다.

In [698]:
show_me_the_list_under_char_and_length(5,'학술대회논문',5)

[['15. [학술대회논문]',
  ' 이경희',
  '2016',
  '현물포트폴리오 및 선물을 이용한 CVaR 기반 ELS 헤징방법',
  '2016 재무금융 관련 5개 학회 학술연구발표회 : 1175 ~ 1208,'],
 ['20. [학술대회논문]',
  ' 장원재',
  '2013',
  '국내 ELS 시장의 성장이 주식시장 변동성에 미치는 영향',
  'KSIAM 학술발표,'],
 ['20. [학술대회논문]',
  ' Luis, J. B.',
  '2001',
  'Expectation and risk aversion measures implicit in prices of options on US and European stock indexes',
  'the Forecasting Financial Market 2001 Conference in London,'],
 ['1. [학술대회논문]',
  ' Agrawal, R.',
  '1993',
  'Efficient Similarity Search in Sequence Databases',
  'Proceedings of the 4th International Conference on Foundations of Data Organization and Algorithms,'],
 ['2. [학술대회논문]',
  ' Agrawal, R.',
  '1995',
  'Fast Similarity Search in the Presence of Noise, Scaling, and Translation in Time-Series Databases',
  'Proceedings of the International Conference on Very Large Databases,']]

In [699]:
[i[1] for i in show_me_the_list_under_char_and_length(5,'학술대회논문') if not i[1]]

[]

In [700]:
[i[-1][:-1] for i in show_me_the_list_under_char_and_length(5,'학술대회논문') if not i[-1][:-1]]

[]

# 길이가 5인 학술대회논문

- 저널명은 큰 문제가 없어보인다.
- 이름이 공백인 경우도 없다.

### 길이가 5인 리스트에서 카테고리가 학술지인 경우
- 학술지가 모양새를 다 갖추기 위해서는,카테고리 , 저자명 , 저널명 , 논문이름 , 이슈 년도가 있어야 한다.
- 아래에서 5개만 끊어서 보면, 어느정도 구색을 맞추고 있는 것으로 보인다. 자세히 살펴보아서 모두 완벽한지 확인해봐야 한다.

In [701]:
show_me_the_list_under_char_and_length(5,'학술지(정기간행물)',5)

[['1. [학술지(정기간행물)]',
  ' 고봉찬',
  '2007',
  '발생액 이상현상에 대한 위험평가',
  'Asia-Pacific Journal of Financial Studies 36 (3) : 425 ~ 461,'],
 ['2. [학술지(정기간행물)]',
  ' 손병철',
  '2005',
  '발생액의 효율적 평가에 영향을 미치는 요인',
  '회계학연구 30 (1) : 59 ~ 91,'],
 ['3. [학술지(정기간행물)]',
  ' Amihud, Yakov',
  '2002',
  'Illiquidity and Stock Returns: Cross-Section and Time- Series Effects',
  'Journal of Financial Markets 5 (1) : 31 ~ 56,'],
 ['4. [학술지(정기간행물)]',
  ' Ball, Ray, S',
  '1995',
  'Problems in Measuring Portfolio Performance: An Application to Contrarian Investment Strategies',
  'Journal of Financial Economics 38 (1) : 79 ~ 107,'],
 ['5. [학술지(정기간행물)]',
  ' Basu, Sudipta',
  '2004',
  'What Do We Learn from Two New Accounting-Based Stock Market Anomalies?',
  'Journal of Accounting and Economics 38 (1) : 333 ~ 348,']]

In [702]:
[i[1] for i in show_me_the_list_under_char_and_length(5,'학술지(정기간행물)') if not i[1]]

['']

In [703]:
[i[-1][:-1] for i in show_me_the_list_under_char_and_length(5,'학술지(정기간행물)') if not i[-1][:-1]]

[]

In [704]:
[i[-1][:-1] for i in show_me_the_list_under_char_and_length(5,'학술지(정기간행물)')][30:40]

['The Accounting Review 77 : 35 ~ 59',
 'Journal of Accounting and Economics 17 : 51 ~ 89',
 'Journal of Accounting Research 29 : 193 ~ 228',
 'Asia-Pacific Journal of Financial Studies 35 (5) : 45 ~ 74',
 'The Journal of Finance LXI : 1035 ~ 1072',
 'The Accounting Review : 201 ~ 244',
 '회계정보연구 21 (4) : 211 ~ 235',
 'Korean Business Review : 103 ~ 125',
 '회계학연구 30 (2) : 277 ~ 306',
 'Journal of Accounting and Economics 17 : 281 ~ 308']

# 길이가 5인 학술지

- 저널명은 큰 문제가 없어보인다.
- 이름이 공백인 경우 1건

### 길이가 5인 리스트에서 카테고리가 학위논문인 경우
- 다른 논문들과는 다르게 길이가 5개이면, 불완전한 참고 문헌 자료이다.
- 현재 이름,년도,논문 이름,학교이름 이 기제되어 있는데, 학위 이름까지 기제되어야 완전한 논문이라고 할 수 있다.

In [705]:
show_me_the_list_under_char_and_length(5,'학위논문',5)

[['8. [학위논문]',
  ' 이송우',
  '2011',
  '민영건강보험의 도덕적 해이와 본인부담금의 도입효과에 관한 연구',
  '국민대학교대학원,'],
 ['12. [학위논문]',
  ' Jung, H',
  '2011',
  'A study on function of corporate group, risk and performance of savings banks',
  'Korea University,'],
 ['5. [학위논문]',
  ' Brennan,F',
  '1986',
  'The Cost of Convenience and the Pricing of Commodity Contingent Claims',
  'University of British Columbia,'],
 ['11. [학위논문]',
  ' Chung, J. W.',
  '2012',
  'Performance Persistence in Private Equity Funds',
  'The Chinese University of Hong Kong,'],
 ['34. [학위논문]',
  ' Li, Y',
  '2009',
  'Three Essays in Empirical Studies on Derivatives',
  'University of Toronto,']]

In [706]:
[i[1] for i in show_me_the_list_under_char_and_length(5,'학위논문') if not i[1]]

[]

In [707]:
[i[-1][:-1] for i in show_me_the_list_under_char_and_length(5,'학위논문') if not i[-1][:-1]]

[]

In [708]:
[i[-1][:-1] for i in show_me_the_list_under_char_and_length(5,'학위논문')][30:40]

[]

# 길이가 5인 학위 논문
- 어자피 불완전한 자료이므로 5개는 날린다.

# # # ## # # # # # # # # # # # #  # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# 길이가 6개인 경우

In [709]:
Counter([re.findall('\[.+\]',i)[0] for i in [i[0] for i in new_df[new_df['length_refer'] == 6]['tuned_refer'].tolist()]])

Counter({'[학위논문]': 94})

### 길이가 6인 리스트에서 카테고리가 단행본인 경우
- 기타 자료가 모양새를 다 갖추기 위해서는,카테고리 , 저자명 , 저널명 , 논문이름 , 이슈 년도가 있어야 한다.
- 아래에서 6개만 끊어서 보면, 어느정도 구색을 맞추고 있는 것으로 보인다. 자세히 살펴보아서 모두 완벽한지 확인해봐야 한다.

### 길이가 6인 리스트에서 카테고리가 학위논문인 경우
- 다른 논문들과는 다르게 길이가 6개이면, 불완전한 참고 문헌 자료이다.
- 현재 이름,년도,논문 이름,학교이름 이 기제되어 있는데, 학위 이름까지 기제되어야 완전한 논문이라고 할 수 있다.

In [710]:
show_me_the_list_under_char_and_length(6,'학위논문',5)

[['10. [학위논문]',
  ' 張榮光',
  '1985',
  '企業合倂과 株式取得이 株主의 富에 미치는 影響에 關한 實證的 硏究',
  '박사',
  '고려대학교 대학원,'],
 ['22. [학위논문]',
  ' Low, A',
  '1992',
  'Essays on asymmetric information in international finance',
  '박사',
  'University of California,'],
 ['32. [학위논문]',
  ' May Hongmei Zhang',
  '2005',
  'The differential effects of proprietary cost on the quality versus quantity of voluntary corporate disclosures',
  'Ph.D',
  'The University of Texas at Austin,'],
 ['11. [학위논문]', ' 이상엽', '2014', '적산법을 이용한 자기자본비용 추정에 관한 연구', '석사', '건국대학교,'],
 ['8. [학위논문]',
  ' 김경용',
  '2013',
  '한달 내의 최대 일별수익률과 주식 기대수익률의 관계',
  '석사',
  '한국과학기술원,']]

In [711]:
[i[1] for i in show_me_the_list_under_char_and_length(6,'학위논문') if not i[1]]

[]

In [712]:
[i[2] for i in show_me_the_list_under_char_and_length(6,'학위논문') if not i[2]]

[]

In [713]:
[i[3] for i in show_me_the_list_under_char_and_length(6,'학위논문') if not i[3]]

[]

In [714]:
[i[4] for i in show_me_the_list_under_char_and_length(6,'학위논문') if not i[4]]

[]

In [715]:
[i[5] for i in show_me_the_list_under_char_and_length(6,'학위논문') if not i[5]]

[]

# 길이가 6인 학위 논문
- 완전한 자료로 판단되므로 모두 딕셔너리에 고려한다.

In [742]:
new_df.tail()

,infor,refer,tuned_refer,length_refer
37242,"Corporate Diversification, Relatedness, and Fi...","40. [기타자료] - Villalonga, Belen / 1999 / Does D...","[40. [기타자료], Villalonga, Belen, 1999, Does Di...",4
37243,"Corporate Diversification, Relatedness, and Fi...","41. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ...","[41. [학술지(정기간행물)], Villalonga, Belen, 2004, D...",5
37244,"Corporate Diversification, Relatedness, and Fi...","42. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ...","[42. [학술지(정기간행물)], Villalonga, Belen, 2004, D...",5
37245,"Corporate Diversification, Relatedness, and Fi...","43. [학술지(정기간행물)] - Weston, J. F / 1970 / The N...","[43. [학술지(정기간행물)], Weston, J. F, 1970, The Na...",5
37246,"Corporate Diversification, Relatedness, and Fi...","44. [학술지(정기간행물)] - Whited, Toni M / 2001 / Is ...","[44. [학술지(정기간행물)], Whited, Toni M, 2001, Is I...",5


# 위의 과정을 토대로 딕셔너리에 포함되어야 하는 데이터프레임의 로우를 지정한다.

결측치 없애주기

In [743]:
nan_index_ls = new_df[new_df['length_refer'] == 1].index.tolist()
#nan_index_ls 는 결측치가 있는 데이터 로우이므로 없애줘야 한다.

In [744]:
new_df.loc[nan_index_ls].shape

(242, 4)

process_df 는 결측치 242개를 제외한 나머지 데이터프레임이다.

In [745]:
process_df = new_df.loc[sorted(list(set(new_df.index).difference(set(nan_index_ls))))]
process_df.tail()

,infor,refer,tuned_refer,length_refer
37242,"Corporate Diversification, Relatedness, and Fi...","40. [기타자료] - Villalonga, Belen / 1999 / Does D...","[40. [기타자료], Villalonga, Belen, 1999, Does Di...",4
37243,"Corporate Diversification, Relatedness, and Fi...","41. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ...","[41. [학술지(정기간행물)], Villalonga, Belen, 2004, D...",5
37244,"Corporate Diversification, Relatedness, and Fi...","42. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ...","[42. [학술지(정기간행물)], Villalonga, Belen, 2004, D...",5
37245,"Corporate Diversification, Relatedness, and Fi...","43. [학술지(정기간행물)] - Weston, J. F / 1970 / The N...","[43. [학술지(정기간행물)], Weston, J. F, 1970, The Na...",5
37246,"Corporate Diversification, Relatedness, and Fi...","44. [학술지(정기간행물)] - Whited, Toni M / 2001 / Is ...","[44. [학술지(정기간행물)], Whited, Toni M, 2001, Is I...",5


길이에 따라 나눠주기

In [746]:
len_5_df = process_df[process_df['length_refer'] == 5].reset_index(drop=True)
len_6_df = process_df[process_df['length_refer'] == 6].reset_index(drop=True) # 모두다 학위논문만 있는 데이터 프레임

길이가 5개인 데이터 프레임 처리하기

In [748]:
len_5_no_name_index_ls = [idx for idx,val in enumerate(len_5_df['tuned_refer']) if not val[1] ]
len(len_5_no_name_index_ls)
# 길이가 5인 참고문헌에서 이름이 없는 경우를 제외시킨다.
# 14는 기타자료의 11 , 단행본의 2 , 학술지의 1이 포함된다.

14

In [749]:
len_5_학위논문_index_ls = [idx for idx,val in enumerate(len_5_df['tuned_refer']) if '학위논문' in val[0]]
len(len_5_학위논문_index_ls)
# 위에서 길이가 5인 참고문헌에서 논문의 특성인 학위 논문일 경우의 5개를 제외한다고 했으므로 적용시킨다.

5

아래의 식으로 process_df 는 우선 결측치가 없고 길이가 5인 논문에서 이름이 없는 것을 제외하였으며, 학위논문 5개를 제외한
데이터 프레임이 된다.

In [751]:
print(len(list(set(len_5_no_name_index_ls).union(len_5_학위논문_index_ls))))
exception_set = set(len_5_no_name_index_ls).union(len_5_학위논문_index_ls)
print(len(len_5_df) - len(exception_set) == len(set(len_5_df.index).difference(exception_set)))

19
True


In [753]:
tuned_len_5_df = len_5_df.loc[sorted(list(set(len_5_df.index).difference(exception_set)))]
tuned_len_5_df.shape

(28821, 4)

# 길이가 5개인 데이터 프레임을 특성에 따라 분류하기

In [763]:
tuned_len_5_df['tuned_refer'].tolist()[:3]

[['1. [학술지(정기간행물)]',
  ' 고봉찬',
  '2007',
  '발생액 이상현상에 대한 위험평가',
  'Asia-Pacific Journal of Financial Studies 36 (3) : 425 ~ 461,'],
 ['2. [학술지(정기간행물)]',
  ' 손병철',
  '2005',
  '발생액의 효율적 평가에 영향을 미치는 요인',
  '회계학연구 30 (1) : 59 ~ 91,'],
 ['3. [학술지(정기간행물)]',
  ' Amihud, Yakov',
  '2002',
  'Illiquidity and Stock Returns: Cross-Section and Time- Series Effects',
  'Journal of Financial Markets 5 (1) : 31 ~ 56,']]

In [760]:
char_ls = [re.findall('\[.+\]',i[0])[0] for i in tuned_len_5_df['tuned_refer']]
char_ls[:10]

['[학술지(정기간행물)]',
 '[학술지(정기간행물)]',
 '[학술지(정기간행물)]',
 '[학술지(정기간행물)]',
 '[학술지(정기간행물)]',
 '[학술지(정기간행물)]',
 '[학술지(정기간행물)]',
 '[학술지(정기간행물)]',
 '[학술지(정기간행물)]',
 '[학술지(정기간행물)]']

In [770]:
name_ls = [i[1] for i in tuned_len_5_df['tuned_refer']]
name_ls[:10]

[' 고봉찬',
 ' 손병철',
 ' Amihud, Yakov',
 ' Ball, Ray, S',
 ' Basu, Sudipta',
 ' Bhardwaj, Ravinder K',
 ' Bhushan, Ravi',
 ' Collins, Daniel W',
 ' Daniel, Kent',
 ' DeBondt, Werner F. M']

In [773]:
year_ls = [i[2] for i in tuned_len_5_df['tuned_refer']]
year_ls[:10]

['2007',
 '2005',
 '2002',
 '1995',
 '2004',
 '1992',
 '1994',
 '2000',
 '1997',
 '1995']

In [777]:
paper_name_ls = [i[3] for i in tuned_len_5_df['tuned_refer']]
paper_name_ls[:10]

['발생액 이상현상에 대한 위험평가',
 '발생액의 효율적 평가에 영향을 미치는 요인',
 'Illiquidity and Stock Returns: Cross-Section and Time- Series Effects',
 'Problems in Measuring Portfolio Performance: An Application to Contrarian Investment Strategies',
 'What Do We Learn from Two New Accounting-Based Stock Market Anomalies?',
 'The January Anomaly: Effects of Low Share Price, Transaction Costs and Bid-Asked Bias',
 'An Informational Efficiency Perspective on the Post- Earnings Announcement Drift',
 'Earnings-Based and Accrual-Based Market Anomalies: One Effect or Two?',
 'Measuring Mutual Fund Performance with Characteristic-Based Benchmarks',
 'Does the Stock Market Overreact?']

In [779]:
journal_ls = [i[4] for i in tuned_len_5_df['tuned_refer']]
journal_ls[:10]

['Asia-Pacific Journal of Financial Studies 36 (3) : 425 ~ 461,',
 '회계학연구 30 (1) : 59 ~ 91,',
 'Journal of Financial Markets 5 (1) : 31 ~ 56,',
 'Journal of Financial Economics 38 (1) : 79 ~ 107,',
 'Journal of Accounting and Economics 38 (1) : 333 ~ 348,',
 'Journal of Finance 47 (2) : 552 ~ 576,',
 'Journal of Accounting and Economics 18 (1) : 45 ~ 66,',
 'Journal of Accounting and Economics 29 (1) : 101 ~ 123,',
 'Journal of Finance 52 (3) : 1035 ~ 1058,',
 'Journal of Finance 40 (3) : 793 ~ 805,']

In [780]:
len_5_final_df = pd.DataFrame()
len_5_final_df['char'] = char_ls
len_5_final_df['name'] = name_ls
len_5_final_df['year'] = year_ls
len_5_final_df['paper_name'] = paper_name_ls
len_5_final_df['journal'] = journal_ls
len_5_final_df

,char,name,year,paper_name,journal
0,[학술지(정기간행물)],고봉찬,2007,발생액 이상현상에 대한 위험평가,Asia-Pacific Journal of Financial Studies 36 (...
1,[학술지(정기간행물)],손병철,2005,발생액의 효율적 평가에 영향을 미치는 요인,"회계학연구 30 (1) : 59 ~ 91,"
2,[학술지(정기간행물)],"Amihud, Yakov",2002,Illiquidity and Stock Returns: Cross-Section a...,"Journal of Financial Markets 5 (1) : 31 ~ 56,"
3,[학술지(정기간행물)],"Ball, Ray, S",1995,Problems in Measuring Portfolio Performance: A...,"Journal of Financial Economics 38 (1) : 79 ~ 107,"
4,[학술지(정기간행물)],"Basu, Sudipta",2004,What Do We Learn from Two New Accounting-Based...,Journal of Accounting and Economics 38 (1) : 3...
5,[학술지(정기간행물)],"Bhardwaj, Ravinder K",1992,The January Anomaly: Effects of Low Share Pric...,"Journal of Finance 47 (2) : 552 ~ 576,"
6,[학술지(정기간행물)],"Bhushan, Ravi",1994,An Informational Efficiency Perspective on the...,Journal of Accounting and Economics 18 (1) : 4...
7,[학술지(정기간행물)],"Collins, Daniel W",2000,Earnings-Based and Accrual-Based Market Anomal...,Journal of Accounting and Economics 29 (1) : 1...
8,[학술지(정기간행물)],"Daniel, Kent",1997,Measuring Mutual Fund Performance with Charact...,"Journal of Finance 52 (3) : 1035 ~ 1058,"
9,[학술지(정기간행물)],"DeBondt, Werner F. M",1995,Does the Stock Market Overreact?,"Journal of Finance 40 (3) : 793 ~ 805,"


# 길이가 6인 데이터 프레임

In [787]:
char_ls1 = [re.findall('\[.+\]',i[0])[0] for i in len_6_df['tuned_refer']]
char_ls1[:10]

['[학위논문]',
 '[학위논문]',
 '[학위논문]',
 '[학위논문]',
 '[학위논문]',
 '[학위논문]',
 '[학위논문]',
 '[학위논문]',
 '[학위논문]',
 '[학위논문]']

In [802]:
name_ls1 = [i[1] for i in len_6_df['tuned_refer']]
name_ls1[:10]

[' 張榮光',
 ' 장영광',
 ' 고유경',
 ' 김영기',
 ' 김태형',
 ' 이남구',
 ' 최선근',
 ' Marchman, B.',
 ' 노성호',
 ' 金俊植']

In [793]:
year_ls1 = [i[2] for i in len_6_df['tuned_refer']]
year_ls1[:10]

['1985',
 '1985',
 '2007',
 '2003',
 '2005',
 '2000',
 '2001',
 '2007',
 '2007',
 '1986']

In [797]:
paper_name_ls1 = [i[3] for i in len_6_df['tuned_refer']]
paper_name_ls1[:10]

['企業合倂과 株式取得이 株主의 富에 미치는 影響에 關한 實證的 硏究',
 '기업합병과 주식취득이 주주 부에 미치는 영향에 관한 실증연구',
 '분석대상 종목군에 신규 편입된 기업에 대한 예측이익의 특성',
 '상호저축은행 도산예측에 관한 연구',
 '시뮬레이션을 이용한 이산 시간 확률 변동성 모형 추정에 관한 연구 : MCMC를 이용한 베이지언 추론 방법을 중심으로',
 '기업의 IR(Investor Relations)에 관한 연구',
 '우리나라 IR 활동이 주가에 미치는 효과에 관한 연구',
 'Reverse Stock Splits: Motivations, Effectiveness, and Stock Price Reactions',
 '코스닥 상장폐지기업의 이익조정에 관한 실증연구',
 '改正商法上의 新株引受權 附社債制度에 관한 硏究']

In [798]:
degree_ls1 = [i[4] for i in len_6_df['tuned_refer']]
degree_ls1[:10]

['박사', '박사', '석사', '박사', '박사', '석사', '박사', '박사', '박사', '석사']

In [800]:
school_ls1 = [i[-1] for i in len_6_df['tuned_refer']]
school_ls1[:10]

['고려대학교 대학원,',
 '고려대학교 대학원,',
 '고려대학교,',
 '성균관대학교 대학원,',
 '서울대학교 대학원,',
 '전남대학교 대학원,',
 '명지대대학원,',
 'Florida State Univ,',
 '대구대학교,',
 '연세대학교 대학원,']

In [803]:
len_6_final_df = pd.DataFrame()
len_6_final_df['char'] = char_ls1
len_6_final_df['name'] = name_ls1
len_6_final_df['year'] = year_ls1
len_6_final_df['paper_name'] = paper_name_ls1
len_6_final_df['degree'] = degree_ls1
len_6_final_df['school'] = school_ls1
len_6_final_df

,char,name,year,paper_name,degree,school
0,[학위논문],張榮光,1985,企業合倂과 株式取得이 株主의 富에 미치는 影響에 關한 實證的 硏究,박사,"고려대학교 대학원,"
1,[학위논문],장영광,1985,기업합병과 주식취득이 주주 부에 미치는 영향에 관한 실증연구,박사,"고려대학교 대학원,"
2,[학위논문],고유경,2007,분석대상 종목군에 신규 편입된 기업에 대한 예측이익의 특성,석사,"고려대학교,"
3,[학위논문],김영기,2003,상호저축은행 도산예측에 관한 연구,박사,"성균관대학교 대학원,"
4,[학위논문],김태형,2005,시뮬레이션을 이용한 이산 시간 확률 변동성 모형 추정에 관한 연구 : MCMC를 이...,박사,"서울대학교 대학원,"
5,[학위논문],이남구,2000,기업의 IR(Investor Relations)에 관한 연구,석사,"전남대학교 대학원,"
6,[학위논문],최선근,2001,우리나라 IR 활동이 주가에 미치는 효과에 관한 연구,박사,"명지대대학원,"
7,[학위논문],"Marchman, B.",2007,"Reverse Stock Splits: Motivations, Effectivene...",박사,"Florida State Univ,"
8,[학위논문],노성호,2007,코스닥 상장폐지기업의 이익조정에 관한 실증연구,박사,"대구대학교,"
9,[학위논문],金俊植,1986,改正商法上의 新株引受權 附社債制度에 관한 硏究,석사,"연세대학교 대학원,"


In [805]:
final_df = pd.concat([len_5_final_df,len_6_final_df],axis=0)
final_df.reset_index(drop=True,inplace=True)
final_df.tail()

,char,degree,journal,name,paper_name,school,year
28910,[학위논문],석사,NaN,김동현,국내 민간의료보험 시장의 역선택에 관한 실증연구,"경희대학교 대학원,",2008
28911,[학위논문],석사,NaN,이수진,민간의료보험이 의료수요에 미치는 영향 : 도덕적 해이와 역선택 효과의 실증적 구분,"연세대학교 대학원,",2012
28912,[학위논문],박사,NaN,한영하,한국주택금융공사 보금자리론 조기상환율 추정 및 가치산정 연구,"연세대학교 대학원,",2014
28913,[학위논문],박사,NaN,"Low, A",Essays on asymmetric information in internatio...,"University of California,",1992
28914,[학위논문],박사,NaN,"Averbukh, V",Pricing American options using Monte Carlo sim...,"Cornell University,",1997


In [813]:
[i for idx,i in enumerate(final_df['paper_name']) if 'Journal' in i]

['Insider Trading, Analysts’ Forecast Revisions, and Earnings Changes, Journal of Accounting',
 'Does herding behavior exist in Chinese stock markets?, Journal of International Financial Markets',
 'The analysis of economic time-series-part i: Prices, Journal of the Royal Statistical Society',
 'Multimarket trading and liquidity: A transaction data analysis of Canada-US interlistings, Journal of International Financial Markets',
 'A general equilibrium approach to monetary theory, Journal of Money',
 'Book value and stock returns, in The Chicago MBA : A Journal of Selected Papers',
 'The investment opportunity set and corporate financing,dividend,and compensation policies.Journal of Financial Economics 32,pp.236-292',
 'Journal of Financial Markets',
 'pp.191-214.(Translated in English)Yu,Il Sung and Yu Tae Lee,2004,Measurement of Financial Risk Using VaR,The Korean Journal of Financial',
 'Journal of Accounting and Economics']

In [814]:
final_df.loc[[idx for idx,i in enumerate(final_df['paper_name']) if 'Journal' in i]]['paper_name'].tolist()

['Insider Trading, Analysts’ Forecast Revisions, and Earnings Changes, Journal of Accounting',
 'Does herding behavior exist in Chinese stock markets?, Journal of International Financial Markets',
 'The analysis of economic time-series-part i: Prices, Journal of the Royal Statistical Society',
 'Multimarket trading and liquidity: A transaction data analysis of Canada-US interlistings, Journal of International Financial Markets',
 'A general equilibrium approach to monetary theory, Journal of Money',
 'Book value and stock returns, in The Chicago MBA : A Journal of Selected Papers',
 'The investment opportunity set and corporate financing,dividend,and compensation policies.Journal of Financial Economics 32,pp.236-292',
 'Journal of Financial Markets',
 'pp.191-214.(Translated in English)Yu,Il Sung and Yu Tae Lee,2004,Measurement of Financial Risk Using VaR,The Korean Journal of Financial',
 'Journal of Accounting and Economics']

In [811]:
final_df.loc[[idx for idx,i in enumerate(final_df['paper_name']) if 'Journal' in i]]

,char,degree,journal,name,paper_name,school,year
1102,[학술지(정기간행물)],NaN,"Auditing and Finance 16 (2) : 167 ~ 187,",Sivakumar. K,"Insider Trading, Analysts’ Forecast Revisions,...",NaN,2001
1384,[학술지(정기간행물)],NaN,"Institutions and Money 16 (2) : 123 ~ 142,","Demirer, R",Does herding behavior exist in Chinese stock m...,NaN,2006
1718,[학술지(정기간행물)],NaN,"Series A 116 : 11 ~ 34,","Kendall, M. G.",The analysis of economic time-series-part i: P...,NaN,1953
5119,[학술지(정기간행물)],NaN,"Institutions and Money 8 : 393 ~ 412,","Foerster, S. R.",Multimarket trading and liquidity: A transacti...,NaN,1998
5142,[학술지(정기간행물)],NaN,"Credit and Banking 1 : 15 ~ 29,","Tobin, J.",A general equilibrium approach to monetary the...,NaN,1969
15674,[단행본],NaN,"University of Chicago : 25 ~ 45,","Stattman, D.","Book value and stock returns, in The Chicago M...",NaN,1980
27962,[학술지(정기간행물)],NaN,"Journal of Financial Economics 32 : 236 ~ 292,","Smith, C",The investment opportunity set and corporate f...,NaN,1992
28141,[학술지(정기간행물)],NaN,"Journal of Financial Markets 6 : 517 ~ 538,","Bae, K. H",Journal of Financial Markets,NaN,2003
28614,[학술지(정기간행물)],NaN,The Korean Journal of Financial Studies 10 (1)...,유일성,"pp.191-214.(Translated in English)Yu,Il Sung a...",NaN,2004
28677,[학술지(정기간행물)],NaN,"Journal of Accounting and Economics,","Abarbanell, J. S",Journal of Accounting and Economics,NaN,1991


In [834]:
journal_counter_ls = [','.join(re.findall('[a-zA-Z가-힣\s]',i)).replace(',','').replace('  ','') for i in final_df['journal'][final_df['journal'].notnull()].tolist()]
dict_ = Counter(journal_counter_ls)
journal_counter_df = pd.DataFrame(index=range(len(list(dict_.keys()))))
journal_counter_df.index = list(dict_.keys())
journal_counter_df['count'] = list(dict_.values())
journal_counter_df.sort_values('count',ascending=False)

,count
Journal of Finance,2870
Journal of Financial Economics,2230
Journal of Finance,1117
재무관리연구,927
재무연구,849
Review of Financial Studies,814
Journal of Financial Economics,740
AsiaPacific Journal of Financial Studies,706
한국증권학회지,618
선물연구,600


In [806]:
final_df.to_csv('well_made_dictionary.csv',index=False)